Larcv has a nice interface for loading data on the fly (including threaded io)

In [1]:
import ROOT
from larcv import larcv
import numpy
import matplotlib.pyplot as plt
%matplotlib inline



Welcome to JupyROOT 6.08/06


In [2]:
from larcv import dataloader2